In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Question 1: Getting started

## Part a

In [ ]:
amp_data = np.load("amp_data.npz")["amp_data"]

In [ ]:
"""Plot a line graph showing the sequence in amp_data"""
def plot_line_graph(data: np.array):
    tt = np.arange(len(data))
    plt.clf()
    plt.plot(tt, data)
    plt.show()
plot_line_graph(amp_data)

In [ ]:
def plot_amplitude_histogram(data, bins=100):
    bins = len(data) / 1000 if (bins == 100) else bins
    data = np.array(data)
    plt.clf()
    plt.hist(data, bins=100)
    plt.show()
plot_amplitude_histogram(amp_data)

## Part b

In [5]:
def split_array(arr):
    size = len(arr)
    train_amount, val_amount, test_amount = 0.7, 0.15, 0.15
    train_split = int(size * train_amount)
    val_split = int(size * val_amount) + train_split
    
    X_shuf_train = arr[:train_split][:, :20]
    y_shuf_train = arr[:train_split][:, 20]
    
    X_shuf_val = arr[train_split:val_split][:, :20]
    y_shuf_val = arr[train_split:val_split][:, 20]
    
    X_shuf_test = arr[val_split:][:, :20]
    y_shuf_test = arr[val_split:][:, 20]
    
    return X_shuf_train, y_shuf_train, X_shuf_val, y_shuf_val, X_shuf_test, y_shuf_test
    

def create_six_arrays(data):
    data = np.array(data)[:, None]
    """remove last values so it can be mapped into a Cx21 matrix"""
    values_to_remove = data.shape[0] % 21
    data = data[:-values_to_remove].reshape(-1, 21)
    """Shuffle the rows of the matrix"""
    np.random.shuffle(data)
    return split_array(data)
    

In [6]:
X_shuf_train, y_shuf_train, X_shuf_val, y_shuf_val, X_shuf_test, y_shuf_test = create_six_arrays(amp_data)

In [7]:
X_shuf_train.shape

(1123775, 20)

# Question 2: Curve fitting

## Part a
Code for a plot that shows 20 training points, a test point, a straight line fit, and a quartic fit.

In [ ]:
step_size = 1/20
xx = np.arange(0, 1, step_size)[:, None]

yy = X_shuf_train[0]
test_point = y_shuf_train[0]

linear_xx = np.hstack([xx, np.ones(xx.shape[0])[:, None]])
linear_weights = np.linalg.lstsq(linear_xx, yy, rcond=None)[0]

quartic_xx = xx ** np.arange(5)
quartic_weights = np.linalg.lstsq(quartic_xx, yy, rcond=None)[0]

"""Add prediction to the plot by using 21 times from 0 to 1.05 with step size 1/20"""
times = np.arange(0, 1 + step_size, step_size)
quartic_xx_with_prediction = times[:,None] ** np.arange(5)
linear_xx_with_prediction = np.vstack([times, np.ones(len(times))]).T

plt.plot(xx, yy, label="Original data")
plt.scatter(1, test_point, color="red", label="Last datapoint")
"""
Change linear/quartic_xx to linear/quartic_xx_with_prediction
and xx to times for getting the prediction in the plot 
"""
plt.plot(xx, np.dot(linear_xx, linear_weights), color="red", label="Straight line fit")
# plt.plot(times, np.dot(linear_xx_with_prediction, linear_weights), color="red", label="Straight line fit")
plt.plot(xx, np.dot(quartic_xx, quartic_weights), color="orange", label="Quartic fit")
# plt.plot(times, np.dot(quartic_xx_with_prediction, quartic_weights), color="orange", label="Quartic fit")

plt.legend()
plt.show()

## Part b

We put equal weight to each residual, no matter how long it happened in the past. Closer datapoints are more important and will lose importance if we look too much into the past. A longer context for the quartic fit will be better as this will prevent overfitting, instead of using just two datapoints.

## Part c

After trying third, fourth and fifth oder polynomial a third order polynomial with a bias fits the data best assuming a context length of 20

# Question 3

## Part b i)

In [13]:
def Phi(C, K):
    times = np.arange(0, 1, step=1/20)[:,None][-C:]
    return times ** np.arange(K)

C = 20
K = 2
design = Phi(C, K)
# print(design)

## Part b ii)

In [8]:
def make_vv(C, K):
    phi_matrix = Phi(C, K)
    phi_t_1 = np.ones((K, 1))
    return phi_matrix @ np.linalg.inv( (phi_matrix.T @ phi_matrix) ).T @ phi_t_1

In [ ]:
def get_prediction_from_v():
    v = make_vv(C, K)
    v.shape
    print(v.T @ X_shuf_train[0])

In [ ]:
get_prediction_from_v()

In [ ]:
def get_linear_prediction():
    times = np.arange(0, 1 + step_size, step_size)
    linear_x = np.vstack([times, np.ones(len(times))]).T
    linear_weights = np.linalg.lstsq(linear_xx, yy, rcond=None)[0]
    print(np.dot(linear_x, linear_weights)[-1])


In [ ]:
def get_quartic_prediction():
    step_size = 1/20
    times = np.arange(0, 1 + step_size, step_size)
    quartic_xx_with_prediction = times[:,None] ** np.arange(5)
    print(np.dot(quartic_xx_with_prediction, quartic_weights)[-1])

## Part b iii)

In [ ]:
step_size = 1/20
xx = np.arange(0, 1, step_size)[:, None]
xx_with_prediction = np.arange(0, 1 + step_size, step_size)[:,None]


def get_linear_prediction_from_v(datapoints: np.array):
    C, K = 20, 2
    v = make_vv(C, K)
    prediction = (v.T @ datapoints)[0]
    # Print prediction by using vector v
    print(f'Linear prediction by using vector v: {prediction}')
    
def get_quartic_prediction_from_v(datapoints: np.array):
    C, K = 20, 5
    v = make_vv(C, K)
    prediction = (v.T @ datapoints)[0]
    # Print prediction by using vector v
    print(f'Quartic prediction by using vector v: {(v.T @ datapoints)[0]}')
    
def get_linear_prediction(datapoints: np.array):
    linear_xx = np.hstack([xx, np.ones(xx.shape[0])[:, None]])
    linear_weights = np.linalg.lstsq(linear_xx, datapoints, rcond=None)[0]
    linear_xx_with_prediction = xx_with_prediction ** np.arange(2)
    # Print linear prediction
    print(f'Linear prediction: {np.dot(linear_xx_with_prediction, linear_weights)[-1]}')

def get_quartic_prediction(datapoints: np.array):
    quartic_xx = xx ** np.arange(5)
    quartic_weights = np.linalg.lstsq(quartic_xx, datapoints, rcond=None)[0]
    quartic_xx_with_prediction = xx_with_prediction ** np.arange(5)
    # Print quartic prediction
    print(f'Quartic prediction: {np.dot(quartic_xx_with_prediction, quartic_weights)[-1]}')



get_linear_prediction_from_v(X_shuf_train[0])
get_linear_prediction(X_shuf_train[0])
get_quartic_prediction_from_v(X_shuf_train[0])
get_quartic_prediction(X_shuf_train[0])


# Question 3c

In [9]:
def get_prediction_from_v(datapoints: np.array, K: int, C: int):
    v = make_vv(C, K)
    prediction = (v.T @ datapoints).T
    # Print prediction by using vector v
    # print(f'Prediction by using vector v: {prediction}')
    return prediction

In [10]:
def mean_square_error(y_true, y_pred):
    return sum((y_true - y_pred)**2) / len(y_true)

In [ ]:
y_pred = get_prediction_from_v(X_shuf_train.T, K=5, C=20)
# y_true = y_shuf_train.reshape(y_pred.shape)
y_true = y_shuf_train[:, None]
print(y_true.shape)

In [ ]:
print(mean_square_error(y_true, y_pred))

In [ ]:
def try_values_for_k_and_C():
    y_true = y_shuf_train[:, None]
    minimum = {'error': float("inf"), 'K':-1, 'C':-1}
    
    def mean_square_error(y_true, y_pred):
        return sum((y_true - y_pred)**2) / len(y_true)

    for C in range(1, 21):
        for K in range(1, 21):
            y_pred = get_prediction_from_v(X_shuf_train.T[-C:], K=K, C=C)
            error = mean_square_error(y_true, y_pred)
            if error < minimum['error']:
                minimum['error'] = error
                minimum['K'] = K
                minimum['C'] = C
                
    print(minimum)

In [14]:
def get_mean_square_error(dataset: np.array, y_true: np.array):
    K, C = 2, 2
    y_pred = get_prediction_from_v(dataset.T[-C:], K, C)
    error = mean_square_error(y_true, y_pred)
    print(error)

In [15]:
get_mean_square_error(X_shuf_train, y_shuf_train[:, None])

Prediction by using vector v: [[-0.00244141]
 [-0.01095581]
 [ 0.1027832 ]
 ...
 [ 0.01971436]
 [-0.0138855 ]
 [-0.03665161]]
[1.35997854e-05]


In [16]:
get_mean_square_error(X_shuf_val, y_shuf_val[:, None])

Prediction by using vector v: [[-0.03182983]
 [ 0.03692627]
 [ 0.01916504]
 ...
 [-0.01831055]
 [ 0.05322266]
 [ 0.00250244]]
[1.34041439e-05]


In [17]:
get_mean_square_error(X_shuf_test, y_shuf_test[:, None])

Prediction by using vector v: [[-0.00448608]
 [ 0.02804565]
 [ 0.0345459 ]
 ...
 [-0.00308228]
 [-0.03033447]
 [ 0.0553894 ]]
[1.33191438e-05]


In [48]:
def question_4a():
    y_true = y_shuf_test[:, None]
    for C in range(1, 21):
        training_error = get_mean_square_error_v2(X_shuf_train, y_shuf_train[:,None], C)[0]
        validation_error = get_mean_square_error_v2(X_shuf_val, y_shuf_val[:,None], C)[0]
        print(f'Training error: {training_error} \t Validation error: {validation_error}')
        
        

In [36]:
def Phi_v2(C):
    return np.arange(0, 1, step=1/20)[:,None][-C:]


In [37]:
def make_vv_v2(C, K=1):
    phi_matrix = Phi_v2(C)
    phi_t_1 = np.ones((K, 1))
    return phi_matrix @ np.linalg.inv( (phi_matrix.T @ phi_matrix) ).T @ phi_t_1

In [38]:
def get_mean_square_error_v2(dataset: np.array, y_true: np.array, C):
    y_pred = get_prediction_from_v_v2(dataset.T[-C:], C)
    error = mean_square_error(y_true, y_pred)
    return error

In [39]:
def get_prediction_from_v_v2(datapoints: np.array, C: int):
    v = make_vv_v2(C)
    prediction = (v.T @ datapoints).T
    # Print prediction by using vector v
    # print(f'Prediction by using vector v: {prediction}')
    return prediction

In [49]:
question_4a()


Training error: 4.3805726298583096e-05 	 Validation error: 4.3155562983714066e-05
Training error: 9.156205675538368e-05 	 Validation error: 9.046011596448667e-05
Training error: 0.00015072534070581978 	 Validation error: 0.0001498783601652463
Training error: 0.00021816882764881304 	 Validation error: 0.00021818975231687534
Training error: 0.00029087980675379687 	 Validation error: 0.0002918703095280602
Training error: 0.0003666200848423686 	 Validation error: 0.00036862895809018805
Training error: 0.0004433117623161807 	 Validation error: 0.0004465939319481955
Training error: 0.0005192667838785514 	 Validation error: 0.0005240006405638587
Training error: 0.0005933912463590216 	 Validation error: 0.0005993573198329325
Training error: 0.0006650349516162934 	 Validation error: 0.0006719638801927429
Training error: 0.0007337772834805714 	 Validation error: 0.0007415405710695369
Training error: 0.0007991010762935496 	 Validation error: 0.0008076319828919448
Training error: 0.000860360612087